In [ ]:
import chess
from stockfish import Stockfish
from ollama import chat
from ollama import ChatResponse
import json
import pandas as pd


In [2]:
stockfish = Stockfish()

In [3]:
csv_file = "/Users/nafis-mac/GitHub Repos/Chess-Research-Project/puzzles_code/filtered_fen.csv"

# Read the first 100 rows of the CSV file
df = pd.read_csv(csv_file, nrows=100)

# Convert the DataFrame rows to a list of lists
fen_notations = df["FEN"].tolist()

print(len(fen_notations))

100


In [9]:
print(fen_notations[0])

rnbqkbnr/p1pppppp/8/1p6/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2


In [4]:
# # URL to scrape the text from.
# url = "https://wtharvey.com/inst.html"

# # Download the text from the website.
# response = requests.get(url)
# if response.status_code == 200:
#     text = response.text
# else:
#     print(f"Failed to retrieve content. Status code: {response.status_code}")
#     text = ""

# # Regular expression pattern for matching FEN strings.
# fen_pattern = re.compile(
#     r'([prnbqkPRNBQK\d/]+\s+[wb]\s+(?:K?Q?k?q?|-)\s+(?:[a-h][36]|-)\s+\d+\s+\d+)'
# )

# # Find all FEN notations in the text.
# fen_notations = fen_pattern.findall(text)

# # Print the extracted FEN notations.
# print("Extracted FEN Notations:")
# # for fen in fen_notations:
# #     print(fen)

# print(fen_notations[:3])

In [5]:
# fen_notations_ten = fen_notations[:10]

# len(fen_notations_ten)

# print(fen_notations_ten[0])

In [6]:
def valid_move_to_string(board):
    legal_moves = [
        board.san(move)
        for move in board.legal_moves
]
    
    moves = str(legal_moves[0])  # Start with the first move

    for move in legal_moves[1:]:  # Skip the first move and iterate over the rest
        moves = moves + " " + move
    return moves

In [22]:
def get_qwen_move(board:chess.Board, fen:str, temp=0.0):
    valid_moves = valid_move_to_string(board)
    
    response: ChatResponse = chat(model='qwen:32b', messages=[
        {
            'role': 'user',
            'content': f"""You are playing chess and it is your turn. This is the current state of the game. Use this to work out where the pieces are on the board:

FEN: {fen}

The possible set of legal moves are: 

Legal Moves: {valid_moves}

You have to choose one from the provided list. Do not choose a move that is not in the list.

Output the best move in SAN format to follow this position. Use the following single blob of JSON. Do not include any other information.
{{
    "san": "The move in SAN format",
    "reason": "Why this is a good move"
}}"""}], options={"temperature": temp})
    
    try:
        parsed_json = json.loads(response.message.content)
        qwen_move = parsed_json["san"]
    except Exception as e:
        print(f"Wrong format given by Qwen. \nQwen Output: {response.message.content}")

    

    return qwen_move

In [13]:
results = []

In [ ]:
puzzle_id = 1


for fen in fen_notations:
    for i in range (50):
        print(f"Processing puzzle_id: {puzzle_id} Iteration: {i+1}\nPosition: {fen}")

        board_stockfish = chess.Board(fen=fen)
        board_qwen = chess.Board(fen=fen)

        assert board_qwen == board_stockfish, "Two boards are not equal"

        stockfish.set_fen_position(fen_position=fen)

        best_move = stockfish.get_best_move()

        board_stockfish.push(chess.Move.from_uci(best_move))
        stockfish.set_fen_position(fen_position=board_stockfish.fen())
        best_eval = stockfish.get_evaluation()  # evaluation in centipawns (or mate score)
        
        # print(f"Stockfish best move: {best_move} with evaluation: {best_eval}")
        if best_eval["type"] == "mate":
            break

        qwen_move = get_qwen_move(board_qwen, fen, 0.6)

        try:
            board_qwen.push_san(qwen_move)
        except Exception as e:
            print(f"Exception captured: {type(e)}")
            continue

        stockfish.set_fen_position(fen_position=board_qwen.fen())
        provided_eval = stockfish.get_evaluation()

        if provided_eval["type"] == "mate":
            # print("ITERATION SKIPPED")
            continue

        score_difference = (best_eval["value"]) - (provided_eval["value"])

        print(f"Stockfish best move: {best_move} with evaluation: {best_eval["value"]}")
        print(f"Qwen move: {qwen_move} with evaluation: {provided_eval["value"]}\n")

        results.append({
            "Iteration": i+1,
            "Puzzle ID": puzzle_id,
            "Method": "Qwen:32B",
            "Best Move Score": best_eval["value"],
            "Qwen Move Score": provided_eval["value"],
            # "Score Difference": score_difference
        })
    
    puzzle_id += 1

Processing puzzle_id: 1 Iteration: 1
Position: rnbqkbnr/p1pppppp/8/1p6/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2
Stockfish best move: f1b5 with evaluation: 195
Qwen move: Nf3 with evaluation: 77

Processing puzzle_id: 1 Iteration: 2
Position: rnbqkbnr/p1pppppp/8/1p6/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2
Stockfish best move: f1b5 with evaluation: 195
Qwen move: Nf3 with evaluation: 77

Processing puzzle_id: 1 Iteration: 3
Position: rnbqkbnr/p1pppppp/8/1p6/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2
Stockfish best move: f1b5 with evaluation: 195
Qwen move: Nf3 with evaluation: 77

Processing puzzle_id: 1 Iteration: 4
Position: rnbqkbnr/p1pppppp/8/1p6/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2
Stockfish best move: f1b5 with evaluation: 195
Qwen move: Nh3 with evaluation: 41

Processing puzzle_id: 1 Iteration: 5
Position: rnbqkbnr/p1pppppp/8/1p6/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2
Stockfish best move: f1b5 with evaluation: 195
Qwen move: Nf3 with evaluation: 77

Processing puzzle_id: 1 Iteration: 6
Position

In [20]:
stockfish.set_fen_position(fen_notations[0])

moves = stockfish.get_top_moves(3)

print(moves)

[{'Move': 'f1b5', 'Centipawn': 183, 'Mate': None}, {'Move': 'd2d4', 'Centipawn': 122, 'Mate': None}, {'Move': 'g1f3', 'Centipawn': 86, 'Mate': None}]


In [ ]:
puzzle_id = 1

for fen in fen_notations_ten:
    for i in range (100):
        if not stockfish.is_fen_valid(fen): exit
        
        print(f"Processing puzzle_id: {puzzle_id} Iteration: {i}\nPosition: {fen}")

        board_stockfish = chess.Board(fen=fen)
        board_qwen = chess.Board(fen=fen)

        stockfish.set_fen_position(fen_position=fen)

        best_move = stockfish.get_best_move()

        board_stockfish.push(chess.Move.from_uci(best_move))
        # best_eval_info = stockfish.get_evaluation()
        best_eval = stockfish.get_evaluation()["value"]  # evaluation in centipawns (or mate score)
        print(f"Stockfish best move: {best_move} with evaluation: {best_eval}")




        qwen_move = get_qwen_move(board_qwen, fen)

        board_qwen.parse_san(qwen_move)

        # chess.Move.from_uci(board_qwen.parse_san(qwen_move))

        # print(f"QWEN MOVE: {qwen_move}\nQWEN MOVE TYPE: {type(qwen_move)}")

        # try:
        #     print(board_qwen.legal_moves)
        #     chess.Move.from_uci(board_qwen.parse_san(qwen_move))
        # except Exception as e:
        #     print(f"Invalid move geenrated by Qwen. The move is: {qwen_move}")
        #     print(f"Exception type: {type(e)}")
        #     continue

        stockfish.set_fen_position(fen_position=board_qwen.fen())
        provided_eval = stockfish.get_evaluation()["value"]

        score_difference = best_eval - provided_eval

        results.append({
            "Iteration": i,
            "Puzzle ID": puzzle_id,
            "Method": "Qwen 32B",
            "Best Move Score": best_eval,
            "Move made by Qwen:32B Score": provided_eval,
            "Score Difference": score_difference
        })
    
    puzzle_id += 1


In [26]:
df = pd.DataFrame(results)
df.to_csv("evaluation_results_qwen32b.csv", index=False)
print("Results saved to evaluation_results.csv")

Results saved to evaluation_results.csv


In [ ]:
f"""You are playing chess and it is your turn.

    This is the current state of the game. Use this to work out where the pieces are on the board:

    FEN: {fen}

    The possible set of legal moves are: Bxe7 Bh6 Bf6 Bh4 Bf4 Be3 Bd2 Bc1 Nxf7 Nd7 Ng6 Nc6 Ng4 Nf3 Nxe7 Nc7+ Nf6+ Nb6 Nf4 Nb4 Ne3 Nc3 Ba6 Bb5+ Bb3 Rg1 Rf1 Kd2 Kf1 Kxd1 Rxd1 Rc1 Rb1 O-O h3 g3 f3 c3 b3 a3 h4 g4 f4 b4 a4. You have to choose one from this list. Do not choose a move that is not in this list.
    
    Output the best move in SAN format to follow this position. Use the following single blob of JSON. Do not include any other information.

    {{
    "san": "The move in SAN format",
    "reason": "Why this is a good move"
    }}
    """

In [25]:
fen = fen_notations[99]

board_test = chess.Board(fen)

board_test.legal_moves

<LegalMoveGenerator at 0x1230a9fa0 (Nh3, Nf3, Ne2, Bd3, Be2, Ke2, Kd2, Qh5, Qg4, Qa4, Qf3, Qd3, Qb3, Qe2, Qd2, Qc2, Bh6, Bg5, Bf4, Be3, Bd2, Nc3, Na3, Nd2, dxe7, d7+, d5, c5, h3, g3, f3, b3, a3, h4, g4, f4, b4, a4)>